In [20]:
user = 'confluent2'
pwd = 'confluent2'
db_name = 'default'
host = 'localhost'
port = '3307'

In [21]:
from sqlalchemy import create_engine, text

# Connect to the database
engine = create_engine(f"mysql+mysqlconnector://{user}:{pwd}@{host}:{port}/{db_name}")

# Test the connection
connection = engine.connect()

In [26]:
# Test the connection
connection = engine.connect()
# Execute a SQL query using text
query = text('SELECT * FROM default.ticker_ethusdt')
# query = text('SHOW TABLES')
result = connection.execute(query)

# Fetch the results
rows = result.fetchall()

# Do something with the fetched data
for row in rows:
    print(row)

# Close the connection
connection.close()


(1703782154046, 2408.0, 1264049116.0, 2382.07, 1264051523.0, 2381.17, 2377.0, 2377.54, 1703782080000.0, 3.63, 0.153, 3463289.547115, 1455.7357, 2379.06478979)


In [23]:
rows

[]

In [51]:
import requests
def get_data(symbol= 'ETHUSDT', tf= '1m'):
    symbol = symbol.upper()
    endpoint = f'https://api.binance.com/api/v3/ticker?symbol={symbol}&windowSize={tf}'
    res = requests.get(endpoint).json()
    res = {k: float(v) for k, v in res.items() if k != 'symbol'}
    
    return res

data = get_data()
data

{'priceChange': -4.2,
 'priceChangePercent': -0.176,
 'weightedAvgPrice': 2380.32846968,
 'openPrice': 2381.49,
 'highPrice': 2382.45,
 'lowPrice': 2376.85,
 'lastPrice': 2377.29,
 'volume': 813.5402,
 'quoteVolume': 1936492.899293,
 'openTime': 1703782140000.0,
 'closeTime': 1703782228613.0,
 'firstId': 1264051343.0,
 'lastId': 1264052745.0,
 'count': 1403.0}

In [25]:
from sqlalchemy import create_engine, Column, Float, Integer, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# Connect to the database
engine = create_engine(f"mysql+mysqlconnector://{user}:{pwd}@{host}:{port}/{db_name}")

# ORM Base
Base = declarative_base()

# ORM Model for ticker_ethusdt
class TickerEthusdt(Base):
    __tablename__ = 'ticker_ethusdt'

    id = Column(Integer, primary_key=True, autoincrement=True)
    closeTime = Column(DateTime, nullable=False)
    count = Column(Float, nullable=False)
    firstId = Column(Float, nullable=False)
    highPrice = Column(Float, nullable=False)
    lastId = Column(Float, nullable=False)
    lastPrice = Column(Float, nullable=False)
    lowPrice = Column(Float, nullable=False)
    openPrice = Column(Float, nullable=False)
    openTime = Column(DateTime, nullable=False)
    priceChange = Column(Float, nullable=False)
    priceChangePercent = Column(Float, nullable=False)
    quoteVolume = Column(Float, nullable=False)
    volume = Column(Float, nullable=False)
    weightedAvgPrice = Column(Float, nullable=False)

# Create the table in the database
Base.metadata.create_all(engine)


data = get_data()
# Create a session
Session = sessionmaker(bind=engine)
session = Session()

# Insert data into the table using ORM
ticker_entry = TickerEthusdt(**data)
session.add(ticker_entry)
session.commit()

# Close the session
session.close()


C:\Users\kimte\AppData\Local\Temp\ipykernel_20632\613826561.py:17: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()
